#  RNN (rekurentne neuralne mreže)

**"A class of neural networks with loops in them, allowing information to persist"**



## Pregled
* RNN
* Motivacija
* Vanila RNN
* LSTM
* Primeri
* Resursi
* Implementacija

## Rekurentne neuralne mreže (RNN)
* Tip NN koji se često koristi kada su ulazni podaci vremenske sekvence tj. kada je umesto **jednog** feature vektora ulaz **sekvenca** feature vektora kroz vreme pri čemu postoje veze između sadašnjosti i prošlosti
* **Primeri**: tekst, zvuk, video, genomi, rukopis, berza...
* Prva mreža koja nije feedforward, sadrži cikluse
* Novi metod treniranja: **backpropagation through time** (BPTT)
* Nekoliko fiksnih arhitektura

## Motivacija
* Nije jasno kako bi standardna NN "uhvatila" pravilnosti kroz vreme
* Kao primer posmatrajmo problem predikcije naredne reči: za ovo je potreban kontekst, tj. nemoguće je izvršiti predikciju na osnovu jedne prethodne reči

## Vanila RNN
* Najjednostavniji tip RNN
* Prva ilustracija sa [colah:Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/): rekurentna veza (grana = težine tj. FC sloj)
* Kako bismo razumeli značenje ove rekurentne veze posmatramo **unrolled** prikaz (druga ilustracija na istom linku) - "odmotali" smo mrežu za fiksan broj vremenskih jedinica
* Sada stvari deluju jasnije: mreža ima svoje **skriveno stanje** koje zadržava kroz vreme i u svakom koraku ga obogaćuje novim ulazom i daje novi izlaz
* To skriveno stanje treba da "zapamti" prošlost i iskoristi je za dati zadatak
* U opštem slučaju u svakom vremenskom koraku imamo novi ulaz i novi izlaz, ali to ne mora biti slučaj (npr. u klasifikaciji zvuka važan nam je samo poslednji izlaz)
* [Još jedna korisna ilustracija koja prikazuje pojedinačne neurone](https://i.imgur.com/yF92R2g.png)
* Na novoj ilustraciji imamo označene tri matrice težina: 
  * $W_{ih}$ - matrica input->hidden, na slici označena kao w1
  * $W_{hh}$ - matrica hidden->hidden, na slici označena kao w2
  * $W_{ho}$ - matrica hidden->output, na slici označena kao w3
* Pored toga ćemo imati dve funkcije aktivacije i dve bias vrednosti:
   * $f_h$ - funkcija aktivacije skrivenog sloja (npr. tanh)
   * $b_h$ - bias skrivenog sloja
   * $f_o$ - funkcija aktivacije izlaznog sloja (npr. sigmoid)
   * $b_o$ - bias izlaznog sloja
* Kao i do sada ulazni vektor označavamo sa $X$, a izlazni sa $Y$
* Skriveno stanje označavamo sa $H$
* Sada su kompletne formule za forward propagation **u jednom vremenskom trenutku** kojima od ulaza i starog skrivenog stanja dobijamo izlaz i novo skriveno stanje $(X, H) \to (H, Y)$:
  * $H_{new} = f_h(W_{ih} \cdot X + W_{hh} \cdot H + b_h) $
  * $Y = f_o(W_{ho} \cdot H + b_o) $
* Što se tiče treninga tj. backpropagation uvodimo metod **backpropagation through time** (BPTT)
  * BPTT koristi identične principe kao standardni BP, tj. propagira gradijente unazad od troška i nije suštinski drukčiji - novo ime koristimo jer je više deskriptivno
* Postoji i **bidirectional RNN** varijanta u kojoj postoje dva skrivena stanja: jedno se propagira kroz vreme unazad a jedno unapred
* **Problem sa vanilla RNN**: ne uspevaju da uhvate pravilnosti kada su sekvence dugačke
  - Jedan od konkretnih problema: u slučaju jako dugih sekvenci BPTT stepenuje matricu $W_{hh}$ dovoljno puta da se gradijenti koji služe za ažuriranje težina izgube (vanishing gradients) ili odu u beskonačno (exploding gradients)
* Zbog ovoga se umesto "klasičnih" RNN koriste naprednije arhitekture
* Najpopularnija takva arhitektura je LSTM 
  
## LSTM (Long Short-Term Memory)
* Pratiti [ilustracije](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) u sekciji *LSTM Networks*
* Umesto jednostavnog pravila za forward propagation koristimo kompleksne **LSTM ćelije**
* Jedna ćelija se sadrži od tri kapije: input gate, forget gate, output gate
* Skriveno stanje je ujedno i izlaz u nekom momentu ($H_t$)
* Pored toga imamo "stanje ćelije" ($C_t$)
* Jednačine:
  * Input gate: $i_t = \sigma(W_i \cdot X_t + U_i \cdot H_{t-1} + b_i)$
  * Forget gate: $f_t = \sigma(W_f \cdot X_t + U_f \cdot H_{t-1} + b_f)$
  * Output gate: $o_t = \sigma(W_o \cdot X_t + U_o \cdot H_{t-1} + b_o)$
  * Dakle svaka kapija je parametrizovana sa dve matrice ($W$ i $U$, pri čemu se to može jednostavno predstaviti kao jedna matrica) i jednom bias vrednošću ($b$)
  * Na osnovu trenutnog ulaza ($X_t$) i prethodnog skrivenog stanja ($H_{t-1}$) svaka kapija daje svoju izlaznu vrednost ($i_t$, odnosno $f_t$, odnosno $o_t$)
  * Pre računanja novog skrivenog stanja tj. izlaza, moramo ažurirati stanje ćelije
  * Za ovo nam je prvo potrebna privremena vrednost zasnovana na novom ulazu koju računamo kao: 
    * $tmp_t = \tanh(W_c \cdot X_t + U_c \cdot H_{t-1} + b_c)$
  * Novo stanje ćelije će biti kombinacija starog stanja ćelije koje "prolazi" kroz forget gate i vrednost $tmp_t$ koja "prolazi" kroz input gate:
    * $C_t = f_t \circ C_{t-1} + i_t \circ tmp_t$
  * Sada računamo novo skriveno stanje tj. izlaz tako što stanje ćelije "prolazi" kroz output gate:
    * $H_t = o_t \circ \tanh(C_t)$

* Pored LSTM često se koristi i GRU (Gated Recurrent Unit)

## Primeri
* [Modeli za predikciju narednog karaktera - generisanje teksta](http://karpathy.github.io/assets/rnn/charseq.jpeg): Šekspir, Wikipedia, Latex, Linux sors kod
* [Još primera korišćenja](https://medium.com/datathings/the-magic-of-lstm-neural-networks-6775e8b540cd): prepoznavanje rukopisa, generisanje rukopisa, generisanje muzike, prevođenje, image captioning...
* Naredni korak: attention?

## Resursi
- [colah: uvod u RNN fokusiran na LSTM, odlične ilustracije](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [karpathy: char-rnn, odličan primer primene](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Implementacija u numpy](https://github.com/rand0musername/psiml2017-workshops/blob/master/4%20-%20RNN/rnn_fwd.py)

## Implementacija RNN (LSTM) u Keras
- Radimo sa IMDB sentiment classification datasetom (nad kojim smo radili Naive Bayes u prvom domaćem)
- Ovaj skup podataka je previše mali da bi LSTM imao prednost nad jednostavnijim metodama, ali ilustruje Keras sintaksu

In [7]:
# Postoji bug u trenutnoj verziji Keras-a zbog kog nije moguce ucitati imdb set
# Jedan od nacina da se ovo privremeno resi je downgrade na verziju numpy 1.16.1
!pip install numpy==1.16.1
import numpy as np

     |████████████████████████████████| 17.3MB 4.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import numpy as np

# Koristimo ovoliko najkoriscenijih reci
max_features = 20000

# Maksimalna duzina neke vremenske sekvence
# U nasem slucaju maksimalan broj reci u nekom review-u
maxlen = 80

# Ostali trening parametri
batch_size = 32
num_epochs = 15

# Ucitavanje podataka
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=max_features)
print(len(X_train))
print(len(X_test))

# Pozivamo funkciju pad_sequences koja pad-uje nase ulazne podatke na duzinu
# maxlen tako da svi budu iste duzine
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print(X_train.shape)
print(X_test.shape)

# Gradimo model
model = Sequential()

# Embedding sloj: mapira svaku rec u vektor duzine 128 koji se takodje uci
# u toku treninga (https://keras.io/layers/embeddings/)
model.add(Embedding(max_features, 128))

# LSTM celija sa izlazom velicine 128
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# Finalni dense sloj kako bi se dobila jedna vrednost, sa sigmoid aktivacijom
model.add(Dense(1, activation='sigmoid'))

# Standardan binarni crossentropy loss i adam optimizacija
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Trening, 15 epoha
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=(X_test, Y_test))
_, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Accuracy na test skupu:', acc)


25000
25000
(25000, 80)
(25000, 80)
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
13952/25000 [===============>..............] - ETA: 1:01 - loss: 0.4934 - acc: 0.7562

KeyboardInterrupt: ignored